# DOG DISEASE DETECTION
## Tim : 
- Christian adi Ananta
- Ezra Abednego

### Dataset : 

### Metode : 




In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing

In [3]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [4]:
import urllib.request
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile
from PIL import Image 
import matplotlib.pyplot as plt
import random
from shutil import copyfile
from tensorflow.keras import regularizers

In [5]:
#Cek File Dataset
print("Number of cat images:",os.listdir('/kaggle/input/skin-disease-dog-augmented/Skin_Disease_Augment'))

Number of cat images: ['earinfection', 'dermatitis', 'skintumors', 'fleas', 'dandruff', 'ringworm']


In [6]:
#Membuat Folder Train Dan Validation
from pathlib import Path
path_img='/kaggle/input/skin-disease-dog-augmented/Skin_Disease_Augment'
img_directory=os.listdir(path_img)
work_path = Path('/kaggle/working/')
for i in ["train","validation"]:
    work_path_parent = Path('/kaggle/working/'+i)
    work_path_parent.mkdir(exist_ok=True)
    for j in img_directory:
        work_path = Path('/kaggle/working/'+i+"/"+j)
        work_path.mkdir(exist_ok=True)

In [7]:
# Split Data dengan presentase 90% Train dan 10% Validation
def split_data(split_size):
    files=[]
    for file_dir in img_directory:
        name_image=[]
        for filename in os.listdir(path_img+"/"+file_dir):
            file=path_img+"/"+file_dir+"/"+filename
            if os.path.getsize(file) > 0:
                name_image.append(filename)
            else:
                print(filename + " is zero length, so ignoring.")
        files.append(name_image)
    for index,path_image in enumerate(files):
        train_len=int(len(path_image)*split_size)
        shuffle=random.sample(path_image,len(path_image))
        train_data=path_image[:train_len]
        validation_data=path_image[train_len:]
        for filename in train_data:
            file=path_img+"/"+img_directory[index]+"/"+filename
            destination="/kaggle/working/train/"+img_directory[index]+"/"+filename
            copyfile(file,destination)
        for filename in validation_data:
            file=path_img+"/"+img_directory[index]+"/"+filename
            destination="/kaggle/working/validation/"+img_directory[index]+"/"+filename
            copyfile(file,destination)
        
split_size=.8
split_data(split_size)

In [8]:
# Melakukan preprocessing data image dari directory

TRAINING_DIR = "/kaggle/working/train"
# Experiment with your own parameters to reach 99.9% validation accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
VALIDATION_DIR = "/kaggle/working/validation/"

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(150, 150))

Found 3599 images belonging to 6 classes.
Found 901 images belonging to 6 classes.


In [9]:
# data_augmentation = tf.keras.Sequential([
#   tf.keras.layers.RandomFlip('horizontal'),
#   tf.keras.layers.RandomRotation(0.2),
# ])

# preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [10]:
pre_trained_model =  tf.keras.applications.InceptionV3(input_shape=(150,150,3),
                                                       include_top=False,
                                                       weights='imagenet')

87910968/87910968 [==============================] - 0s 0us/step


In [11]:
#freeze layer
for layer in pre_trained_model.layers:
    layer.trainable = False

In [12]:
#get last layer
last_layer =  pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [13]:
#Add Dense Layer
x = layers.LayerNormalization()(last_output)
x = layers.Flatten()(x)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

In [14]:
#Train Model
checkpoint_filepath = "/kaggle/working/checkpoint"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(train_generator,
                  epochs=100,
                  validation_data=validation_generator, 
                callbacks=[model_checkpoint_callback])

Epoch 1/100
113/113 [==============================] - 108s 921ms/step - loss: 1.1361 - accuracy: 0.5696 - precision: 0.7017 - recall: 0.4418 - val_loss: 0.5882 - val_accuracy: 0.7714 - val_precision: 0.8421 - val_recall: 0.6748
Epoch 2/100
113/113 [==============================] - 93s 824ms/step - loss: 0.6644 - accuracy: 0.7491 - precision: 0.8076 - recall: 0.6894 - val_loss: 0.4990 - val_accuracy: 0.8091 - val_precision: 0.8588 - val_recall: 0.7558
Epoch 3/100
113/113 [==============================] - 93s 822ms/step - loss: 0.5215 - accuracy: 0.8072 - precision: 0.8428 - recall: 0.7655 - val_loss: 0.4652 - val_accuracy: 0.8135 - val_precision: 0.8448 - val_recall: 0.7913
Epoch 4/100
113/113 [==============================] - 94s 833ms/step - loss: 0.4454 - accuracy: 0.8336 - precision: 0.8663 - recall: 0.8047 - val_loss: 0.3132 - val_accuracy: 0.8835 - val_precision: 0.9105 - val_recall: 0.8690
Epoch 5/100
113/113 [==============================] - 71s 623ms/step - loss: 0.3537 - 

In [15]:
model.evaluate(train_generator)
model.evaluate(validation_generator)

29/29 [==============================] - 11s 393ms/step - loss: 0.1553 - accuracy: 0.9623 - precision: 0.9644 - recall: 0.9612


[0.15534178912639618,
 0.9622641801834106,
 0.9643652439117432,
 0.9611542820930481]

In [16]:
class_map = train_generator.class_indices
classes_disease = []
for key in class_map.keys():
    classes_disease.append(key)
print(classes_disease)

['dandruff', 'dermatitis', 'earinfection', 'fleas', 'ringworm', 'skintumors']


In [17]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

for i in os.listdir('/kaggle/working/validation'):
    for j in os.listdir('/kaggle/working/validation/'+i):
        path='/kaggle/working/validation/'+i+'/'+j
        img = load_img(path, target_size=(150, 150))
        x = img_to_array(img)
        x /= 255
        x = np.expand_dims(x, axis=0)
        image_tensor = np.vstack([x])
        classes = model.predict(image_tensor)
        print(j)
        print(classes[0])
        print(classes_disease[np.argmax(classes[0])])
        break

        
# path = '/kaggle/working/validation/ringworm/ringworm_dark_40.png'
# img = load_img(path, target_size=(150, 150))
# x = img_to_array(img)
# x /= 255
# x = np.expand_dims(x, axis=0)
# image_tensor = np.vstack([x])
# classes = model.predict(image_tensor)
# print(classes[0])
# print(np.argmax(classes[0]))
# print(classes_disease[np.argmax(classes[0])])

1/1 [==============================] - 1s 999ms/step
skintumors_dark_181.png
[1.6381193e-12 1.8913893e-06 9.1918417e-16 4.4890380e-11 3.9307542e-06
 9.9999416e-01]
skintumors
1/1 [==============================] - 0s 39ms/step
fleas_flips_53.png
[2.8243997e-07 6.2327252e-17 6.7277234e-18 9.9999976e-01 3.7760871e-17
 2.3990292e-16]
fleas
1/1 [==============================] - 0s 39ms/step
earinfection_flips_153.png
[3.6641727e-16 2.7581249e-19 1.0000000e+00 1.0550631e-22 7.7407756e-21
 1.9306151e-19]
earinfection
1/1 [==============================] - 0s 41ms/step
dandruff_flips_271.png
[1.0000000e+00 4.0523235e-18 2.4745253e-21 1.2910326e-10 5.5642463e-16
 1.5635893e-10]
dandruff
1/1 [==============================] - 0s 39ms/step
lickgranuloma35 - Copy (2).png
[5.2640161e-19 1.0000000e+00 2.5227624e-14 7.6545466e-14 1.5537763e-09
 9.6775556e-12]
dermatitis
1/1 [==============================] - 0s 39ms/step
ringworm_flips_218.png
[1.6670269e-12 9.8546523e-05 2.1099640e-10 1.0439861e-1

In [18]:
model.save("model_saved.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
